<a href="https://colab.research.google.com/github/byiringiroscar/NLP-Data-science-ML-and-Artificial-Intelligence/blob/main/spams_filter_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import string
import pandas as pd

In [3]:
df = pd.read_csv('/content/emails.csv')
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [4]:
df['spam'].value_counts()

spam
0    4360
1    1368
Name: count, dtype: int64

In [7]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


def preprocess(text):
  # remove punctuation and lowercase
  text = "".join([t.lower() for t in text if t not in string.punctuation])
  # tokenize
  tokens = text.split(" ") # 'color printing' == ['color', 'printing']

  # filter out stopwords
  return " ".join(t for t in tokens if t not in ENGLISH_STOP_WORDS)


df['text'] = df['text'].apply(lambda text: preprocess(text))

In [8]:
df.head()

,text,spam
0,subject naturally irresistible corporate ident...,1
1,subject stock trading gunslinger fanny merril...,1
2,subject unbelievable new homes easy im wantin...,1
3,subject 4 color printing special request addi...,1
4,subject money software cds software compati...,1


In [11]:
X = df['text']
Y = df['spam']

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer()
tfidf.fit(X)
X_vectors = tfidf.transform(X)

In [14]:
X_vectors

<5728x37023 sparse matrix of type '<class 'numpy.float64'>'
	with 508390 stored elements in Compressed Sparse Row format>

SPLITTING DATA INTO Train and Test sets

In [15]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X_vectors, Y, test_size=0.2, random_state=42
)


print(X_train.shape)
print(y_train.shape)

(4582, 37023)
(4582,)


**Training KNN CLASSIFIER**

In [16]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier()

Predict some instances

In [17]:
y_predi1 = knn_classifier.predict(X_test)
y_predi1

array([0, 0, 0, ..., 1, 0, 0])

In [18]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

# Accuracy, precision ,  recall scores

print("Accuracy:", accuracy_score(y_test, y_predi1))
print("Precision:", precision_score(y_test, y_predi1))
print("Recall:", recall_score(y_test, y_predi1))

Accuracy: 0.9729493891797557
Precision: 0.9814126394052045
Recall: 0.9103448275862069


Saving model to a file

In [20]:
from joblib import dump


dump(knn_classifier, 'spam_model.joblib')
dump(tfidf, 'tfidf.joblib')

['tfidf.joblib']